<a href="https://colab.research.google.com/github/ykato27/Scraping/blob/main/Scraping_%E4%BE%A1%E6%A0%BC_com.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 価格.comからレンズ情報をスクレイピング

In [1]:
!pip install yapf isort autopep8

In [2]:
# 自動コード整形
import autopep8
import isort
from IPython.core.magic import register_cell_magic
from yapf.yapflib.yapf_api import FormatCode


@register_cell_magic
def max_fmt(line, cell):
   """
   My formatter cell magic comannd.
   Please install autopep8, isort and yapf before using this magic command.
   !pip install autopep8 isort yapf
   """
   ret = isort.code(cell)
   ret = autopep8.fix_code(ret)
   print(FormatCode(ret, style_config='pep8')[0])

In [3]:
# %%max_fmt
import datetime
import re
import time
import urllib.request

import requests
from bs4 import BeautifulSoup

In [4]:
# %%max_fmt
# 対象のサイトURL
# 価格.com レンズランキング
url = "https://kakaku.com/camera/camera-lens/ranking_1050/"
page_count = 1
linklist = []
# ランキングの全ページから各レンズのページを取得
while True:
    category_res = requests.get(url + "?page=" + str(page_count)).text
    soup = BeautifulSoup(category_res, 'html.parser')  # BeautifulSoupの初期化
    print("{} ページ目".format(page_count))
    for elm in soup.find_all("a"):
        if 'href' in elm.attrs:
            link_url = elm.attrs['href']
            if "https://kakaku.com/item/" in link_url:
                linklist.append(link_url)

    # print(link_url)
    # 次ページがなくなるまでのフラグ
    a_next_tag = soup.find_all("li", {"class": "next"})
    if a_next_tag:
        # if page_count < 1:
        page_count += 1
        continue
    break
# 重複を削除
linklist = sorted(list(set(linklist)), key=linklist.index)

1 ページ目
2 ページ目
3 ページ目
4 ページ目
5 ページ目
6 ページ目
7 ページ目
8 ページ目
9 ページ目
10 ページ目
11 ページ目
12 ページ目
13 ページ目
14 ページ目
15 ページ目
16 ページ目
17 ページ目
18 ページ目
19 ページ目
20 ページ目
21 ページ目
22 ページ目
23 ページ目
24 ページ目
25 ページ目
26 ページ目
27 ページ目
28 ページ目
29 ページ目
30 ページ目


In [5]:
# %%max_fmt
# 書き込みファイル名（取得日時）
now = datetime.datetime.now()
filename = "renzu" + now.strftime('%Y%m%d_%H%M%S') + '.csv'
f = open(filename, 'a', encoding='cp932', errors='ignore')
f.write("カメラ名,順位,最低価格,クレジット最低価格,価格URL,")

# レンズのスペックのシートに移動
page_html = linklist[0] + "spec/#tab"
res = urllib.request.urlopen(page_html)
page_soup = BeautifulSoup(res, 'html.parser')

# レンズのスペックのテーブルを取得
table = page_soup.findAll("table", {"class": "tblBorderGray mTop15"})[0]
rows = table.findAll("tr")
index = -1

# レンズのスペックのテーブルの各見出しを書き込み
for row in rows:
    csvRow = []
    for cell in row.findAll('th'):
        index += 1
        if index == 0:
            continue
        if index == 17:
            continue
        if index == 26:
            continue
        if index == 29:
            continue
        cell = cell.get_text()
        cell = re.sub(r"[\n\t\s]*", "", str(cell))
        f.write(cell)
        f.write(",")
f.write("\n")

# レンズの価格情報を書き込み
for page_url in linklist:
    page_html = page_url + "spec/#tab"
    res = urllib.request.urlopen(page_html)
    page_soup = BeautifulSoup(res, 'html.parser')

    # 必要な要素とclass名
    name = page_soup.find("h2", itemprop="name").text
    try:
        rank = page_soup.find("span", class_="rankNum").text
    except AttributeError:
        rank = ''
    try:
        low_price = page_soup.find("div", class_="priceWrap").find(
            "span", class_="priceTxt").text
        low_price = low_price.replace(',', '')
    except AttributeError:
        low_price = ''
    try:
        cre_price = page_soup.find("div", class_="creditCard").find(
            "span", class_="priceTxt").text
        cre_price = cre_price.replace(',', '')
    except AttributeError:
        cre_price = ''
    print(rank)
    print(low_price)
    f.write(name)
    f.write(",")
    f.write(rank)
    f.write(",")
    f.write(low_price)
    f.write(",")
    f.write(cre_price)
    f.write(",")
    f.write(page_url)
    f.write(",")

    # レンズの情報を書き込み
    # テーブルを指定
    table = page_soup.findAll("table", {"class": "tblBorderGray mTop15"})[0]
    rows = table.findAll("tr")

    # テーブルの書き込み
    for row in rows:
        csvRow = []
        for cell in row.findAll('td'):
            cell = cell.get_text()
            cell = re.sub(r"[\n\t\s]*", "", str(cell))
            f.write(cell)
            f.write(",")
    f.write("\n")
f.close()

 1
¥89100
 2
¥116820
 3
¥138105
 4
¥331650
 5
¥306900
 6
¥251460
 7
¥143490
 8
¥192875
 9
¥272250
 10
¥69576
 11
¥31637
 12
¥46530
 13
¥118800
 14
¥76230
 15
¥139400
 16
¥25500
 17
¥137470
 18
¥280000
 19
¥289575
 20
¥94864
 21
¥59678
 22
¥116322
 23
¥249800
 24
¥374220
 25
¥89700
 25
¥35630
 27
¥14710
 27
¥221680
 29
¥79000
 30
¥89100
 31
¥108000
 32
¥62700
 33
¥59690
 33
¥185800
 33
¥170900
 36
¥22770
 36
¥86785
 36
¥201670
 39
¥258000
 40
¥436660
 41
¥253800
 42
¥76471
 42
¥102960
 44
¥97400
 44
¥136140
 46
¥26800
 47
¥94314
 47
¥126720
 49
¥111800
 49
¥170000
 51
¥25020
 52
¥163350
 53
¥176713
 53
¥234645
 55
¥66000
 55
¥78388
 57
¥140998
 57
¥268936
 59
¥52800
 59
¥792000
 61
¥230540
 62
¥220000
 62
¥104980
 62
¥113210
 65
¥188928
 65
¥82644
 65
¥271800
 68
¥75800
 68
¥67680
 68
¥89445
 68
¥253000
 72
¥74250
 73
¥88000
 74
¥65000
 75
¥129217
 75
¥66790
 75
¥45099
 78
¥113997
 79
¥31720
 79
¥47120
 81
¥108000
 81
¥142000
 83
¥155438
 83
¥42800
 83
¥54870
 83
¥99800
 87
¥65980
 87
¥